In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm
import numpy as np

In [6]:
# =====================================================
# 1️⃣ Data Preparation
# =====================================================
transform = transforms.Compose([
    transforms.Lambda(lambda img: img.convert("RGB")),  # <--- Convert grayscale to RGB
    transforms.Resize((224, 224)),           # 1️⃣ Resize images
    transforms.ToTensor(),                   # 2️⃣ Convert to PyTorch tensor
    transforms.Normalize(                    # 3️⃣ Normalize pixel values
        mean=[0.485, 0.456, 0.406],          # ImageNet means
        std=[0.229, 0.224, 0.225]            # ImageNet std deviations
    )
])

data_dir = "./data"
dataset = datasets.Caltech101(data_dir, transform=transform, download=True)

100%|████████████████████████████████████████| 137M/137M [01:17<00:00, 1.78MB/s]


In [8]:
# Check classes
all_classes = dataset.categories  # list of class names (strings)
print("Total classes in dataset:", len(all_classes))
print("class names are :", all_classes[:])

Total classes in dataset: 101
class names are : ['Faces', 'Faces_easy', 'Leopards', 'Motorbikes', 'accordion', 'airplanes', 'anchor', 'ant', 'barrel', 'bass', 'beaver', 'binocular', 'bonsai', 'brain', 'brontosaurus', 'buddha', 'butterfly', 'camera', 'cannon', 'car_side', 'ceiling_fan', 'cellphone', 'chair', 'chandelier', 'cougar_body', 'cougar_face', 'crab', 'crayfish', 'crocodile', 'crocodile_head', 'cup', 'dalmatian', 'dollar_bill', 'dolphin', 'dragonfly', 'electric_guitar', 'elephant', 'emu', 'euphonium', 'ewer', 'ferry', 'flamingo', 'flamingo_head', 'garfield', 'gerenuk', 'gramophone', 'grand_piano', 'hawksbill', 'headphone', 'hedgehog', 'helicopter', 'ibis', 'inline_skate', 'joshua_tree', 'kangaroo', 'ketch', 'lamp', 'laptop', 'llama', 'lobster', 'lotus', 'mandolin', 'mayfly', 'menorah', 'metronome', 'minaret', 'nautilus', 'octopus', 'okapi', 'pagoda', 'panda', 'pigeon', 'pizza', 'platypus', 'pyramid', 'revolver', 'rhino', 'rooster', 'saxophone', 'schooner', 'scissors', 'scorpion'

In [10]:
# Split 80/20
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [18]:
# =====================================================
# 2️⃣ Define AlexNet Convolutional Layers (Manual)
# =====================================================
class AlexNetFeatures(nn.Module):
    def __init__(self):
        super(AlexNetFeatures, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(96, 256, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(256, 384, kernel_size=3, padding=1),
            nn.ReLU()
        )
        self.layer4 = nn.Sequential(
            nn.Conv2d(384, 384, kernel_size=3, padding=1),
            nn.ReLU()
        )
        self.layer5 = nn.Sequential(
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )

    def forward(self, x):
        x = self.layer1(x)
        print("After layer1 (Conv1):", x.shape)  # (N, 96, 54, 54)
        x = self.layer2(x)
        print("After layer2 (Conv2):", x.shape)  # (N, 256, 26, 26)
        x = self.layer3(x)
        print("After layer3 (Conv3):", x.shape)  # (N, 384, 26, 26)
        x = self.layer4(x)
        print("After layer4 (Conv4):", x.shape)  # (N, 384, 26, 26)
        x = self.layer5(x)
        print("After layer5 (Conv5):", x.shape)  # (N, 256, 12, 12)
        return x

In [20]:
# Instantiate feature extractor
alexnet_features = AlexNetFeatures()
alexnet_features.eval()

AlexNetFeatures(
  (layer1): Sequential(
    (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (layer4): Sequential(
    (0): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (layer5): Sequential(
    (0): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
)

In [22]:
# =====================================================
# 3️⃣ Extract Features
# =====================================================
def extract_features(dataloader, model):
    features, labels = [], []
    with torch.no_grad():
        for inputs, targets in tqdm(dataloader, desc="Extracting features"):
            outputs = model(inputs)
            # Pool & flatten
            outputs = F.adaptive_avg_pool2d(outputs, (6, 6))
            outputs = outputs.view(outputs.size(0), -1)
            features.append(outputs.cpu().numpy())
            labels.append(targets.numpy())
    return np.concatenate(features), np.concatenate(labels)


In [24]:
X_train, y_train = extract_features(train_loader, alexnet_features)
X_test, y_test = extract_features(test_loader, alexnet_features)

print("Train features:", X_train.shape)
print("Test features:", X_test.shape)


Extracting features:   0%|                              | 0/217 [00:00<?, ?it/s]

After layer1 (Conv1): torch.Size([32, 96, 27, 27])
After layer2 (Conv2): torch.Size([32, 256, 13, 13])
After layer3 (Conv3): torch.Size([32, 384, 13, 13])


Extracting features:   0%|                      | 1/217 [00:00<02:16,  1.58it/s]

After layer4 (Conv4): torch.Size([32, 384, 13, 13])
After layer5 (Conv5): torch.Size([32, 256, 6, 6])
After layer1 (Conv1): torch.Size([32, 96, 27, 27])


Extracting features:   1%|▏                     | 2/217 [00:00<01:31,  2.35it/s]

...

After layer1 (Conv1): torch.Size([32, 96, 27, 27])
After layer2 (Conv2): torch.Size([32, 256, 13, 13])
After layer3 (Conv3): torch.Size([32, 384, 13, 13])
After layer4 (Conv4): torch.Size([32, 384, 13, 13])
After layer5 (Conv5): torch.Size([32, 256, 6, 6])


Extracting features:  98%|█████████████████████▌| 54/55 [00:15<00:00,  3.38it/s]

After layer1 (Conv1): torch.Size([32, 96, 27, 27])
After layer2 (Conv2): torch.Size([32, 256, 13, 13])
After layer3 (Conv3): torch.Size([32, 384, 13, 13])
After layer4 (Conv4): torch.Size([32, 384, 13, 13])
After layer5 (Conv5): torch.Size([32, 256, 6, 6])


Extracting features: 100%|██████████████████████| 55/55 [00:15<00:00,  3.63it/s]

After layer1 (Conv1): torch.Size([8, 96, 27, 27])
After layer2 (Conv2): torch.Size([8, 256, 13, 13])
After layer3 (Conv3): torch.Size([8, 384, 13, 13])
After layer4 (Conv4): torch.Size([8, 384, 13, 13])
After layer5 (Conv5): torch.Size([8, 256, 6, 6])
Train features: (6941, 9216)
Test features: (1736, 9216)


In [26]:
# =====================================================
# 4️⃣ Standardize Features
# =====================================================
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [28]:
# =====================================================
# 5️⃣ Train and Evaluate Multiple ML Models
# =====================================================
# Logistic Regression
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
acc_lr = accuracy_score(y_test, y_pred_lr)


In [30]:
# Random Forest
rf = RandomForestClassifier(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
acc_rf = accuracy_score(y_test, y_pred_rf)


In [32]:
# SVM
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)
acc_svm = accuracy_score(y_test, y_pred_svm)

In [40]:
# =====================================================
# 6️⃣ Report Results
# =====================================================
print("\n================ OUTPUT SHAPES & ACCURACIES ================")
print(f"Logistic Regression Accuracy : {acc_lr*100:.2f}%")
print(f"Random Forest Accuracy       : {acc_rf*100:.2f}%")
print(f"SVM Accuracy (linear)        : {acc_svm*100:.2f}%")

print("\nDetailed Classification Report (LR):")
print(classification_report(y_test, y_pred_lr))


================ OUTPUT SHAPES & ACCURACIES ================
Logistic Regression Accuracy : 65.03%
Random Forest Accuracy       : 45.68%
SVM Accuracy (linear)        : 61.81%

Detailed Classification Report (LR):
              precision    recall  f1-score   support

           0       0.86      0.99      0.92        90
           1       0.96      0.96      0.96       110
           2       0.79      0.89      0.84        38
           3       0.95      1.00      0.98       177
           4       1.00      0.57      0.73         7
           5       0.92      0.95      0.94       162
           6       0.25      0.11      0.15         9
           7       0.00      0.00      0.00         4
           8       0.00      0.00      0.00        14
           9       0.00      0.00      0.00        12
          10       0.50      0.33      0.40         9
          11       0.75      0.33      0.46         9
          12       0.50      0.67      0.57        21
          13       0.63      